## Setup & EDA

We first need to install all the needed packages

In [1]:
!pip install bitsandbytes
!pip install transformers[torch]
!pip install transformers accelerate
!pip install --upgrade transformers

**Restart kernel after installation**

We will then import all the needed libraries

In [2]:
import json
from pathlib import Path
from datasets import Dataset,DatasetDict, load_dataset
import pandas as pd

We reformatted the data set now so that we can use a standard load_dataset

In [3]:
data = load_dataset("json", data_files="/notebooks/data/train1.json", field="data")

Using custom data configuration default-2cdb10d3c3013ad5
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-2cdb10d3c3013ad5/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/1 [00:00<?, ?it/s]

lets just display the dataset to check

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 11
    })
})

Great, lest split it up, and create validation and training datasets

In [5]:
split_datasets = data["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")
split_datasets

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/json/default-2cdb10d3c3013ad5/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-b31d2a285b5688c4.arrow and /root/.cache/huggingface/datasets/json/default-2cdb10d3c3013ad5/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-2908dd47d7b61a9a.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 9
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 2
    })
})

In [6]:
split_datasets["train"][1]["translation"]

{'ktl': 'fun main (args: {{type0}} <{{type1}}>)',
 'pli': 'PROCEDURE MAIN {{type0}} {{type1}}'}

## Tokenize

Import a tokenizer to convert all our inputs and targets

TOKENs in compiler and LLM/Transformers have different meanings. Tokenisation here refer to the creation of a numerical value for the input tokens, a vector that can be used by the model.

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
from transformers import AutoTokenizer

model_checkpoint = "meta-llama/CodeLlama-7b-hf" # Replace this with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt", use_auth_token=True, force_download=True)

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

Note because we merged the dataset, we can actually use the same dataset for both source and target.
The most interesting thing here is the tokenizer, a hugging face function, that produces the attention_mask.

In [11]:
pli_sentence = split_datasets["train"][1]["translation"]["pli"]
ktl_sentence = split_datasets["train"][1]["translation"]["ktl"]

inputs = tokenizer(pli_sentence, return_tensors="pt")
targets = tokenizer(ktl_sentence, return_tensors="pt")
inputs, targets

({'input_ids': tensor([[    1, 13756, 29907,  3352, 11499, 14861,  1177,  8620,  1853, 29900,
            930,  8620,  1853, 29896,   930]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[    1,  2090,  1667,   313,  5085, 29901,  8620,  1853, 29900,   930,
            529,  6224,  1853, 29896,   930, 12948]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])})

In [14]:
wrong_targets = tokenizer(ktl_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))

A quick function to clean up the input sequence and set the model up to accept the input sequence

overflowing_tokens and num_truncated tokens are things like whitespace, and start of sequence/end of sequence etc.

This model expects the inputs to be named "labels".


In [15]:
max_length = 64

def preprocess_function(examples):
    inputs = [ex["pli"] for ex in examples["translation"]]
    targets = [ex["ktl"] for ex in examples["translation"]]
    
    # Tokenize inputs and targets separately
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding=True)
    model_targets = tokenizer(targets, max_length=max_length, truncation=True, padding=True)
    
    # Remove unnecessary keys from model_inputs
    model_inputs.pop("overflowing_tokens", None)
    model_inputs.pop("num_truncated_tokens", None)
    
    # Add targets to model_inputs
    model_inputs["labels"] = model_targets["input_ids"]
    
    return model_inputs

In [16]:
tokenizer.pad_token = tokenizer.eos_token

In [17]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-2cdb10d3c3013ad5/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-c6d9dd215773c873.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-2cdb10d3c3013ad5/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-d21a019601bf3b25.arrow


## Fine-tuning model

In [15]:
pip install --upgrade bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_checkpoint, force_download=True)

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # No evaluation during training
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=150,
    predict_with_generate=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=None,  # Assuming no evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [ ]:
import os

# Disable wandb
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train()